In [75]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

# Make dummy data
data = pd.DataFrame(np.random.randint(0, 2, size=(10, 8)), columns=list('ABCDEFGH'))
data

,A,B,C,D,E,F,G,H
0,1,1,0,1,1,0,1,0
1,0,0,0,0,1,0,1,0
2,0,1,0,0,0,1,1,0
3,1,0,0,0,1,0,0,0
4,0,1,1,1,1,0,0,1
5,0,0,1,0,1,0,1,1
6,1,0,1,0,0,0,1,1
7,1,1,1,1,1,1,1,0
8,1,1,0,0,1,1,0,1
9,0,0,0,1,1,0,0,1


In [76]:
# Count the number of 0s and 1s in each column
# The number of 1s is the number of times each item appears
value_counts = data.apply(pd.value_counts)
value_counts

,A,B,C,D,E,F,G,H
0,5,5,6,6,2,7,4,5
1,5,5,4,4,8,3,6,5


In [77]:
value_counts['A'][1]

5

Using the lecture notes explanation of the Apriori Algorithm, we have 4 steps to do.
1. Candidate Generation
2. Candidate Pruning
3. Support Counting
4. Candidate Elimination

Sample code for 1 and 2 itemset

Define the min support

In [176]:
min_support = 4

In [274]:
# Combined dictionary of frequent itemsets
combined_freq_itemsets = {}

Generate F1 (frequent 1-itemsets)

In [275]:
# Get the frequent itemsets with count greater than or equal to min_support
columns = data.columns
frequent_itemsets = {}
for column in columns:
    # Append the itemset and its count to the dictionary if the count is greater than or equal to min_support
    if value_counts[column][1] >= min_support:
        frequent_itemsets[column] = value_counts[column][1]
        # frequent_itemsets.append((column, value_counts[column][1]))
        # data.drop(column, axis=1, inplace=True)

print(frequent_itemsets)

dummy_dict = frequent_itemsets.copy()
for key, item in dummy_dict.copy().items():
    dummy_dict[(tuple(key))] = dummy_dict.pop(key)
print(dummy_dict)
    
combined_freq_itemsets.update(dummy_dict)

{'A': 5, 'B': 5, 'C': 4, 'D': 4, 'E': 8, 'G': 6, 'H': 5}
{('A',): 5, ('B',): 5, ('C',): 4, ('D',): 4, ('E',): 8, ('G',): 6, ('H',): 5}


Step 1: Candidate Generation

In [276]:
# Generate all possible combinations of frequent itemsets with k+1 items
combinations = []
k = 1
combinations.append(list(itertools.combinations(frequent_itemsets.keys(), k+1)))

combinations

[[('A', 'B'),
  ('A', 'C'),
  ('A', 'D'),
  ('A', 'E'),
  ('A', 'G'),
  ('A', 'H'),
  ('B', 'C'),
  ('B', 'D'),
  ('B', 'E'),
  ('B', 'G'),
  ('B', 'H'),
  ('C', 'D'),
  ('C', 'E'),
  ('C', 'G'),
  ('C', 'H'),
  ('D', 'E'),
  ('D', 'G'),
  ('D', 'H'),
  ('E', 'G'),
  ('E', 'H'),
  ('G', 'H')]]

Step 2: Candidate Pruning (do not need to prune for 2 itemset as F1 items are all frequent)

Step 3: Support Counting

In [277]:
# Convert the list of lists of tuples to a list of tuples
combinations = combinations[0]

In [278]:
# Count the number of occurences of each combination in the data
combinations_count = {}
for combination in combinations:
    # Using groupby and size to count the number of occurences of each combination
    # Resetting the index to get the count of each combination as a column in the dataframe
    test = data.groupby(list(combination)).size().reset_index(name='count')

    # Append the combination and its count to the dictionary
    # The count of each combination is the last value in the count column
    # as the last row of the dataframe is when both items are present in one transaction in the original data dataframe
    combinations_count[combination] = test['count'].iloc[-1]

# print(test)
combinations_count

{('A', 'B'): 3,
 ('A', 'C'): 2,
 ('A', 'D'): 2,
 ('A', 'E'): 4,
 ('A', 'G'): 3,
 ('A', 'H'): 2,
 ('B', 'C'): 2,
 ('B', 'D'): 3,
 ('B', 'E'): 4,
 ('B', 'G'): 3,
 ('B', 'H'): 2,
 ('C', 'D'): 2,
 ('C', 'E'): 3,
 ('C', 'G'): 3,
 ('C', 'H'): 3,
 ('D', 'E'): 4,
 ('D', 'G'): 2,
 ('D', 'H'): 2,
 ('E', 'G'): 4,
 ('E', 'H'): 4,
 ('G', 'H'): 2}

In [279]:
# test.index.values[-1].count(1)
test1 = test
count = test1['count'].iloc[-1]
count

2

Step 4: Candidate Elimination

In [280]:
# Prune the combinations with count less than min_support
for combination in combinations_count.copy().keys():
    if combinations_count[combination] < min_support:
        combinations_count.pop(combination)

print(combinations_count)
combined_freq_itemsets.update(combinations_count)

{('A', 'E'): 4, ('B', 'E'): 4, ('D', 'E'): 4, ('E', 'G'): 4, ('E', 'H'): 4}


Candidate generation for 2 or more frequent itemsets

In [281]:
# Merge the combinations if the first k-1 items are the same
# and the last item is different
# This is done to generate combinations with k+1 items
# from combinations with k items

# Compare first k-1 items of each combination
# If they are the same, merge them
# If they are not the same, do not merge them
# The merged combinations are stored in a dictionary
merged_combinations = {}
# for combination1 in combinations_count.keys():
#     for combination2 in combinations_count.keys():
#         # Check if the first k-1 items are the same
#         if combination1[:-1] == combination2[:-1]:
#             # Check if the last item is different
#             if combination1[-1] != combination2[-1]:
#                 # Merge the combinations
#                 merged_combinations[combination1 + (combination2[-1],)] = 0

for index, combination1 in enumerate(combinations_count.keys()):
    for combination2 in list(combinations_count.keys())[index+1:]:
        # Check if the first k-1 items are the same
        if combination1[:-1] == combination2[:-1]:
            # Check if the last item is different
            if combination1[-1] != combination2[-1]:
                # Merge the combinations
                merged_combinations[combination1 + (combination2[-1],)] = 0


merged_combinations


{('E', 'G', 'H'): 0}

Support counting

In [282]:
# Count the number of occurences of each combination in the data
merged_combinations_count = {}
for combination in merged_combinations.keys():
    # Using groupby and size to count the number of occurences of each combination
    # Resetting the index to get the count of each combination as a column in the dataframe
    test = data.groupby(list(combination)).size().reset_index(name='count')

    # Append the combination and its count to the dictionary
    # The count of each combination is the last value in the count column
    # as the last row of the dataframe is when both items are present in one transaction in the original data dataframe
    merged_combinations_count[combination] = test['count'].iloc[-1]

# print(test)
merged_combinations_count

{('E', 'G', 'H'): 1}

In [283]:
# Prune the combinations with count less than min_support
for combination in merged_combinations_count.copy().keys():
    if merged_combinations_count[combination] < min_support:
        merged_combinations_count.pop(combination)

print(merged_combinations_count)
combined_freq_itemsets.update(merged_combinations_count)


{}


In [284]:
combined_freq_itemsets 

{('A',): 5,
 ('B',): 5,
 ('C',): 4,
 ('D',): 4,
 ('E',): 8,
 ('G',): 6,
 ('H',): 5,
 ('A', 'E'): 4,
 ('B', 'E'): 4,
 ('D', 'E'): 4,
 ('E', 'G'): 4,
 ('E', 'H'): 4}

Part 2: Rule generation

In [285]:
# Generate rules for frequent itemsets with k+1 items with min confidence
# The rules are generated by splitting the combination into two parts
min_confidence = 0.5
rules = {}
for key in combined_freq_itemsets.keys():
    # Split the combination into two parts
    # The first part is the antecedent and the second part is the consequent
    for i in range(1, len(key)):
        antecedent = key[:i]
        consequent = key[i:]

        # Calculate the confidence of the rule
        # Confidence = support of combination / support of antecedent
        confidence = combined_freq_itemsets[key] / combined_freq_itemsets[antecedent]

        # Check if the confidence is greater than min_confidence
        if confidence > min_confidence:
            # Append the rule to the rules dictionary
            rules[(antecedent, consequent)] = confidence

rules

{(('A',), ('E',)): 0.8, (('B',), ('E',)): 0.8, (('D',), ('E',)): 1.0}

In [286]:
# Prune smaller rules based on confidence of larger rules
# If larger rule has confidence less than min_confidence, smaller rules are pruned

# Sort the rules in descending order of confidence
sorted_rules = sorted(rules.items(), key=lambda x: x[1], reverse=True)
sorted_rules

# Prune the rules
pruned_rules = {}
for rule in sorted_rules:
    # Append the rule to the pruned_rules dictionary if it is not a subset of any rule in the dictionary
    if not any([set(rule[0]).issubset(set(pruned_rule[0])) for pruned_rule in pruned_rules.keys()]):
        pruned_rules[rule[0]] = rule[1]

pruned_rules

{(('D',), ('E',)): 1.0, (('A',), ('E',)): 0.8, (('B',), ('E',)): 0.8}

In [312]:
# Apriori algorithm
# We combine the above steps to generate frequent itemsets with k+1 items
# from frequent itemsets with k items
# We continue this process until we get no frequent itemsets with k+1 items
# We then combine the frequent itemsets with k items to generate association rules
# We continue this process until we get no association rules
# We then combine the association rules to generate association rules with k+1 items


# Function to generate frequent itemsets with 1 item (initialisation)
def generate_freq_1_itemsets(data, min_support, combined_freq_itemsets):
    # Count the number of 0s and 1s in each column
    # The number of 1s is the number of times each item appears
    value_counts = data.apply(pd.value_counts)

    # Get the frequent itemsets with count greater than or equal to min_support
    columns = data.columns
    frequent_itemsets = {}
    for column in columns:
        # Append the itemset and its count to the dictionary if the count is greater than or equal to min_support
        if value_counts[column][1] >= min_support:
            frequent_itemsets[column] = value_counts[column][1]
            # frequent_itemsets.append((column, value_counts[column][1]))
            # data.drop(column, axis=1, inplace=True)

    dummy_dict = frequent_itemsets.copy()
    for key, item in dummy_dict.copy().items():
        dummy_dict[(tuple(key))] = dummy_dict.pop(key)

    combined_freq_itemsets.update(dummy_dict)

    print(frequent_itemsets)
    return frequent_itemsets


# Function to generate frequent itemsets with k+1 items
def generate_k_plus_1_candidate_itemsets(frequent_itemsets, k):
    # Generate all possible combinations of frequent itemsets with k+1 items

    # If k = 1, we do not need to merge the combinations
    if k == 1:
        combinations = []
        combinations.append(list(itertools.combinations(frequent_itemsets.keys(), k+1)))
        return combinations
    
    else:
        # Merge the combinations if the first k-1 items are the same
        # and the last item is different
        # This is done to generate combinations with k+1 items
        # from combinations with k items
        # Compare first k-1 items of each combination
        # If they are the same, merge them
        # If they are not the same, do not merge them
        # The merged combinations are stored in a dictionary
        merged_combinations = {}
        

        for index, combination1 in enumerate(frequent_itemsets.keys()):
            for combination2 in list(frequent_itemsets.keys())[index+1:]:
                # Check if the first k-1 items are the same
                if combination1[:-1] == combination2[:-1]:
                    # Check if the last item is different
                    if combination1[-1] != combination2[-1]:
                        # Merge the combinations
                        merged_combinations[combination1 + (combination2[-1],)] = 0

    
        return merged_combinations

# Function to count the number of occurences of each combination in the candidate itemsets
def k_plus_1_itemsets_support_counting(k_plus_1_candidate_itemsets, k):
    # If k = 1, we need to convert the list of lists of tuples to a list of tuples
    if k == 1:
        k_plus_1_candidate_itemsets = k_plus_1_candidate_itemsets[0]

    # Count the number of occurences of each combination in the data
    candidate_itemsets_count = {}
    for candidate_itemset in k_plus_1_candidate_itemsets:
        # Using groupby and size to count the number of occurences of each combination
        # Resetting the index to get the count of each combination as a column in the dataframe
        test = data.groupby(list(candidate_itemset)).size().reset_index(name='count')

        # Append the combination and its count to the dictionary
        # The count of each combination is the last value in the count column
        # as the last row of the dataframe is when both items are present in one transaction in the original data dataframe
        candidate_itemsets_count[candidate_itemset] = test['count'].iloc[-1]

    return candidate_itemsets_count


def candidate_elimination(combinations_count, min_support, combined_freq_itemsets):
    
    # Prune the combinations with count less than min_support
    for combination in combinations_count.copy().keys():
        if combinations_count[combination] < min_support:
            combinations_count.pop(combination)
    
    combined_freq_itemsets.update(combinations_count)
    return combinations_count

def generate_rules(combined_freq_itemsets, min_confidence):
    # Generate rules for frequent itemsets with k+1 items with min confidence
    # The rules are generated by splitting the combination into two parts
    rules = {}
    for key in combined_freq_itemsets.keys():
        # Split the combination into two parts
        # The first part is the antecedent and the second part is the consequent
        for i in range(1, len(key)):
            antecedent = key[:i]
            consequent = key[i:]

            # Calculate the confidence of the rule
            # Confidence = support of combination / support of antecedent
            confidence = combined_freq_itemsets[key] / combined_freq_itemsets[antecedent]

            # Check if the confidence is greater than min_confidence
            if confidence > min_confidence:
                # Append the rule to the rules dictionary
                rules[(antecedent, consequent)] = confidence

    return rules
    

In [319]:
def apriori(data, min_support, min_confidence):
    
    # Combined dictionary of frequent itemsets
    combined_freq_itemsets = {}

    # Get frequent 1 itemsets
    frequent_1_itemsets = generate_freq_1_itemsets(data, min_support, combined_freq_itemsets)

    k_plus_1_candidate_itemsets = None
    k_plus_1_itemsets_support_count = None
    k_plus_1_frequent_itemsets = None
    
    k = 1

    while True:
        # print(k)
        if k == 1:
            k_plus_1_candidate_itemsets = generate_k_plus_1_candidate_itemsets(frequent_1_itemsets, k)
        else:
            k_plus_1_candidate_itemsets = generate_k_plus_1_candidate_itemsets(k_plus_1_frequent_itemsets, k)
        
        # print(k_plus_1_candidate_itemsets)
        k_plus_1_itemsets_support_count = k_plus_1_itemsets_support_counting(k_plus_1_candidate_itemsets, k)
        
        k_plus_1_frequent_itemsets = candidate_elimination(k_plus_1_itemsets_support_count, min_support, combined_freq_itemsets)
        # print(k_plus_1_frequent_itemsets)
        k += 1
        
        # If there are no frequent itemsets with k+1 items, break
        if len(k_plus_1_frequent_itemsets) == 0:
            break

    # Generate rules for frequent itemsets with k+1 items with min confidence
    # The rules are generated by splitting the combination into two parts
    rules = generate_rules(combined_freq_itemsets, min_confidence)
    
    return combined_freq_itemsets, rules


combined_freq_itemsets, rules = apriori(data, 4, 0.5)

{'A': 5, 'B': 5, 'C': 4, 'D': 4, 'E': 8, 'G': 6, 'H': 5}
Combined frequent itemsets {('A',): 5, ('B',): 5, ('C',): 4, ('D',): 4, ('E',): 8, ('G',): 6, ('H',): 5, ('A', 'E'): 4, ('B', 'E'): 4, ('D', 'E'): 4, ('E', 'G'): 4, ('E', 'H'): 4}


In [321]:
print('combined frequent itemsets: ', combined_freq_itemsets)

combined frequent itemsets:  {('A',): 5, ('B',): 5, ('C',): 4, ('D',): 4, ('E',): 8, ('G',): 6, ('H',): 5, ('A', 'E'): 4, ('B', 'E'): 4, ('D', 'E'): 4, ('E', 'G'): 4, ('E', 'H'): 4}


In [328]:
for key, item in rules.items():
    for i in range(1, len(key)):
        antecedent = key[:i]
        consequent = key[i:]
        print('antecedent: ', list(sum(antecedent, ())), 'consequent: ', list(sum(consequent, ())), 'confidence: ', item)

antecedent:  ['A'] consequent:  ['E'] confidence:  0.8
antecedent:  ['B'] consequent:  ['E'] confidence:  0.8
antecedent:  ['D'] consequent:  ['E'] confidence:  1.0
